In this notebook we will look into creating multiple models and fine tuning them to get the best possible results.

I will be using the following models:
1. BaggingClassifier
2. RandomForestClassifier
3. ExtraTreesClassifier
4. VotingClassifier (with TBD models)
5. BayesianGaussianMixture
6. GaussianNB
7. GaussianMixture
8. KNNeighborsClassifier

The choise of the models are not based on any specific reason, but rather to try out different models and see how they perform. There is although one condition, the model needs to be able to give a probability output, as this will be used in order to give a confidence score.

In [1]:
from sklearn.ensemble import BaggingClassifier # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier
from sklearn.ensemble import RandomForestClassifier #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier
from sklearn.mixture import BayesianGaussianMixture #https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html
from sklearn.naive_bayes import GaussianNB #https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB
from sklearn.mixture import GaussianMixture #https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture
from sklearn.neighbors import NearestNeighbors #https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier